In [ ]:
import nltk
from nltk.util import ngrams
from collections import Counter
import string
import math

# Gerekli nltk verilerini indirme
nltk.download('punkt')

# Metin dosyasını okuma
with open("odtu_derlemi.txt", "r", encoding="utf-8") as file:
    metin = file.read()

# Metni kelime seviyesinde tokenize et
kelimeler = nltk.word_tokenize(metin.lower())

# Noktalama işaretlerinden kurtulma
kelimeler = [kelime for kelime in kelimeler if kelime not in string.punctuation]

# 2-gram (Bigram) modelini oluştur
bigramlar = list(ngrams(kelimeler, 2))

# Bigramların frekansını hesapla
bigram_freq = Counter(bigramlar)

# Unigramların frekansını hesapla
unigram_freq = Counter(kelimeler)

# Toplam unigram ve bigram sayısını al
toplam_unigram = sum(unigram_freq.values())
toplam_bigram = sum(bigram_freq.values())

# Verilen bir cümlenin bigram zinciri olasılığını hesaplama
def cumlenin_olasiligini_hesapla(cumle):
    # Cümleyi tokenize et ve noktalama işaretlerinden kurtul
    kelimeler = nltk.word_tokenize(cumle.lower())
    kelimeler = [kelime for kelime in kelimeler if kelime not in string.punctuation]
    
    # İlk kelimenin unigram olasılığı
    ilk_kelime = kelimeler[0]
    if ilk_kelime in unigram_freq:
        olasilik = unigram_freq[ilk_kelime] / toplam_unigram
    else:
        olasilik = 1 / toplam_unigram  # Eğer ilk kelime yoksa çok küçük bir olasılık ver
    
    # Sonraki kelimeler için bigram olasılıklarını çarp
    for i in range(1, len(kelimeler)):
        onceki_kelime = kelimeler[i - 1]
        simdiki_kelime = kelimeler[i]
        bigram = (onceki_kelime, simdiki_kelime)
        
        if bigram in bigram_freq:
            olasilik *= bigram_freq[bigram] / unigram_freq[onceki_kelime]
        else:
            olasilik *= 1 / toplam_bigram  # Eğer bigram yoksa çok küçük bir olasılık ver
    
    return olasilik

In [ ]:
# Test cümlesi
cumle = "hafta sonu sinemaya gidelim"
olasilik = cumlenin_olasiligini_hesapla(cumle)

print(f"'{cumle}' cümlesinin olasılığı: {olasilik}")

Yukarıda Markov zinciri formülü yardımıyla bigram'leri kullanarak cümle olasılıklarını hesapladık. Ancak bigram olasılıklarını çarptığımız için daha uzun cümlelerin olasılıkları daha düşük olacaktır. Bu handikaptan kurtulmak için log-olasılıkları hesaplayalım:

In [ ]:
# Verilen bir cümlenin logaritmik bigram zinciri olasılığını hesaplama
def cumlenin_log_olasiligini_hesapla(cumle):
    # Cümleyi tokenize et ve noktalama işaretlerinden kurtul
    kelimeler = nltk.word_tokenize(cumle.lower())
    kelimeler = [kelime for kelime in kelimeler if kelime not in string.punctuation]
    
    # İlk kelimenin unigram olasılığı
    ilk_kelime = kelimeler[0]
    if ilk_kelime in unigram_freq:
        log_olasilik = math.log(unigram_freq[ilk_kelime] / toplam_unigram)
    else:
        log_olasilik = math.log(1 / toplam_unigram)  # Eğer ilk kelime yoksa çok küçük bir olasılık ver
    
    # Sonraki kelimeler için bigram log olasılıklarını toplama
    for i in range(1, len(kelimeler)):
        onceki_kelime = kelimeler[i - 1]
        simdiki_kelime = kelimeler[i]
        bigram = (onceki_kelime, simdiki_kelime)
        
        if bigram in bigram_freq:
            log_olasilik += math.log(bigram_freq[bigram] / unigram_freq[onceki_kelime])
        else:
            log_olasilik += math.log(1 / toplam_bigram)  # Eğer bigram yoksa çok küçük bir olasılık ver
    
    return log_olasilik

In [ ]:
# Test için bir dizi cümle
cumleler = [
    "hafta sonu sinemaya gidelim",
    "hafta sonu sinemaya gidersek",
    "hafta sonu sinemaya gittik",
    "hafta sonu sinemaya gideceğiz"
]

# Her bir cümle için logaritmik olasılığı hesapla ve yazdır
for cumle in cumleler:
    log_olasilik = cumlenin_log_olasiligini_hesapla(cumle)
    print(f"'{cumle}' cümlesinin logaritmik olasılığı: {log_olasilik}")

herhangi bir ikilinin ikili sıklık listesinde olup olmadığına bakabiliriz, listede olmayan ikililer cümlenin olasılığını düşürür: 

In [ ]:
bigram = ("sinemaya", "gidelim")
        
if bigram in bigram_freq:
    print("var")
else:
    print("yok")

n-gram dil modellerini kullanarak bir kelimeden sonra gelebilecek en olası kelimeyi bulabiliriz:

In [ ]:
def sonraki_kelimeyi_tahmin_et(baslangic_kelimesi, bigram_freq):
    olasi_bigramlar = {bigram: freq for bigram, freq in bigram_freq.items() if bigram[0] == baslangic_kelimesi}
    en_olasi_bigram = max(olasi_bigramlar, key=olasi_bigramlar.get) if olasi_bigramlar else None
    if en_olasi_bigram:
        return en_olasi_bigram[1]
    return None

baslangic = "üniversite"
sonraki_kelime = sonraki_kelimeyi_tahmin_et(baslangic, bigram_freq)
print(f"'{baslangic}' kelimesinden sonra en olası kelime: {sonraki_kelime}")


In [ ]:
def metin_olustur(baslangic_kelime, bigram_freq, n=10):
    metin = [baslangic_kelime]
    for _ in range(n):
        sonraki_kelime = sonraki_kelimeyi_tahmin_et(metin[-1], bigram_freq)
        if sonraki_kelime:
            metin.append(sonraki_kelime)
        else:
            break
    return ' '.join(metin)

baslangic = "üniversite"
uretilen_metin = metin_olustur(baslangic, bigram_freq)
print(f"Oluşturulan metin: {uretilen_metin}")

Metin üretiminde özellikle deterministik yaklaşımlar kullanıldığında (yani her zaman en yüksek olasılıklı kelimenin seçilmesi) sürekli aynı kelime dizisine ulaşılabilir. Bundan kaçınmak için aşağıdaki gibi rastsallık eklenebilir:

In [ ]:
import random
def sonraki_kelimeyi_rastsal_tahmin_et(baslangic_kelimesi, bigram_freq):
    olasi_bigramlar = {bigram: freq for bigram, freq in bigram_freq.items() if bigram[0] == baslangic_kelimesi}
    
    if not olasi_bigramlar:
        return None
    
    bigramlar = list(olasi_bigramlar.keys())
    frekanslar = list(olasi_bigramlar.values())
    
    # Bigram olasılıklarını normalize etme
    toplam_frekans = sum(frekanslar)
    olasiliklar = [freq / toplam_frekans for freq in frekanslar]
    
    # Olasılıklara dayalı rastgele seçim yap
    secilen_bigram = random.choices(bigramlar, weights=olasiliklar, k=1)[0]
    return secilen_bigram[1]

# Rastsal kelime seçimi ile metin oluşturma
def metin_olustur_rastsal(baslangic_kelime, bigram_freq, n=10):
    metin = [baslangic_kelime]
    for _ in range(n):
        sonraki_kelime = sonraki_kelimeyi_rastsal_tahmin_et(metin[-1], bigram_freq)
        if sonraki_kelime:
            metin.append(sonraki_kelime)
        else:
            break
    return ' '.join(metin)

# Test için metin oluşturma
baslangic = "üniversite"
uretilen_metin = metin_olustur_rastsal(baslangic, bigram_freq)
print(f"Oluşturulan metin: {uretilen_metin}")

N-gram modelleri yazım hatası olan kelimeleri düzeltebilir. Bir kelime yanlış yazıldığında, model bir kelimenin en çok hangi kelimeyle eşleştiğini bulabilir ve hatalı kelime yerine doğrusunu önerir.

In [ ]:
# Hatalı bir kelimeyi düzeltmek için benzer kelimeleri bulma (Levenshtein mesafesi veya basit bir yakınlık ölçüsü kullanarak)
def kelime_duzeltme(hatali_kelime, unigram_freq):
    # Levenshtein mesafesi (hamming distance gibi) kullanmak daha gelişmiş olabilir, ancak burada basit bir benzerlik ölçüsü kullanıyoruz.
    def benzerlik(kelime1, kelime2):
        # Aynı uzunluktaki kelimeler daha olasıdır
        if abs(len(kelime1) - len(kelime2)) > 2:
            return 0
        # Harflerin ne kadarının eşleştiğine bakarak basit bir benzerlik skoru verelim
        return sum(1 for a, b in zip(kelime1, kelime2) if a == b)

    # Tüm unigramlar içinde hatalı kelimeye en çok benzeyen kelimeleri bulalım
    en_benzer_kelime = max(unigram_freq.keys(), key=lambda kelime: benzerlik(hatali_kelime, kelime))
    return en_benzer_kelime

# Test cümlesi: Hatalı bir kelime ile
hatali_cumle = "yarın hava sırak olacak"
cumledeki_kelime = "sırak"

# Düzeltme önerisi
duzeltilmis_kelime = kelime_duzeltme(cumledeki_kelime, unigram_freq)

print(f"Hatalı kelime: {cumledeki_kelime}, Düzeltme önerisi: {duzeltilmis_kelime}")

İmla düzeltme işleminde önceki ve sonraki kelimeyi hesaba katarsak model çok daha güçlü hale gelir:
(Ayrıca iki kelime arasındaki harflerin birebir eşleşmesine bakarak basit bir benzerlik skoru hesaplama yerine Levenshtein mesafesini kullanalım)

In [ ]:
# ilgili kütüphaneyi yüklemek için bu kodu bir kez çalıştırmalıyız:
!pip install python-Levenshtein

In [ ]:
import Levenshtein
# Hatalı bir kelimeyi düzelten fonksiyon (bağlama duyarlı)
def kelime_duzeltme_bağlam(hatali_kelime, onceki_kelime, sonraki_kelime, unigram_freq, bigram_freq):
    # Basit benzerlik ölçümü: Aynı uzunluktaki kelimeler daha olasıdır, harf eşleşmelerine bakıyoruz
    def benzerlik(kelime1, kelime2):
        if abs(len(kelime1) - len(kelime2)) > 2:
            return 0
        return sum(1 for a, b in zip(kelime1, kelime2) if a == b)

    # Hataya en benzer kelimeleri bulalım
    #olasi_duzeltmeler = sorted(unigram_freq.keys(), key=lambda kelime: benzerlik(hatali_kelime, kelime), reverse=True)[:50]
    olasi_duzeltmeler = sorted(unigram_freq.keys(), key=lambda kelime: Levenshtein.distance(hatali_kelime, kelime))[:50]

    # Bigram olasılıklarını hesaba katarak en olası kelimeyi seç
    en_iyi_kelime = None
    en_yuksek_olasilik = 0

    for duzeltme in olasi_duzeltmeler:
        # Önceki ve sonraki kelimelerle bigram oluşturma
        bigram_olasilik = 1
        if onceki_kelime and (onceki_kelime, duzeltme) in bigram_freq:
            bigram_olasilik *= bigram_freq[(onceki_kelime, duzeltme)] / unigram_freq[onceki_kelime]
        else:
            bigram_olasilik *= 0.0001  # Eğer bigram yoksa küçük bir olasılık verelim
        
        if sonraki_kelime and (duzeltme, sonraki_kelime) in bigram_freq:
            bigram_olasilik *= bigram_freq[(duzeltme, sonraki_kelime)] / unigram_freq[duzeltme]
        else:
            bigram_olasilik *= 0.0001  # Eğer bigram yoksa küçük bir olasılık verelim

        # En yüksek olasılığa sahip kelimeyi seç
        if bigram_olasilik > en_yuksek_olasilik:
            en_yuksek_olasilik = bigram_olasilik
            en_iyi_kelime = duzeltme
        #print(duzeltme + " " + str(bigram_olasilik))
    return en_iyi_kelime

# Test cümlesi: Hatalı bir kelimeyle birlikte
hatali_cumle = "yarın hava sırak olacak"
cumle_tokens = nltk.word_tokenize(hatali_cumle.lower())

# Hatalı kelime ve çevresindeki kelimeler
target = 2
hatali_kelime = cumle_tokens[target]
onceki_kelime = cumle_tokens[target-1]
sonraki_kelime = cumle_tokens[target+1]

# Düzeltme önerisi
duzeltilmis_kelime = kelime_duzeltme_bağlam(hatali_kelime, onceki_kelime, sonraki_kelime, unigram_freq, bigram_freq)

print(f"Hatalı kelime: {hatali_kelime}, Düzeltme önerisi: {duzeltilmis_kelime}")